# Assignment B: Back to the moon

It's been so long that we went to the moon, that we've forgetten how.  Your mission is to investigate the orbital mechanics we'll need to do it again.  Because the equations are quite a mess, we'll develop numerical methods for a simple pendulum you have in your garage, test them, and once they work properly, apply them to the space missions towards the end.

You look carefully at your pendulum, have a little think, and decide that:

$$m \ddot{\phi} = - \frac{mg}{l}\sin(\phi) - \alpha l \dot\phi |l \dot\phi|$$

is probably a pretty good model - where $\phi(t)$ is the deflection angle in time, $g$ the gravitational accelertion, $l$ the length of the pendulum and $\alpha$ the drag factor due to air resistance. Using the auxillary variables
$$
u_0 := \phi, \qquad
u_1 := \dot{\phi} 
$$
transform the ODE into a 1st-order system of the form $\dot u = f(u)$.  Your first job is to implement $f$ and $\partial f/\partial u$:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from time import time

In [ ]:
l   = 1.            # (m) Length of pendulum
g   = 9.81          # (m/s^2)
m   = 0.001         # (kg) Mass of the bob
rho = 1.225         # (kg/m^3) Density of air
r   = 0.05          # (m) Radius of sphere
A   = np.pi * r**2  # (m^2) Cross-secional area
c_d = 0.47          # (.) Drag coefficient of a sphere

alpha = c_d * rho * A / 2.   # Initially assume air-resitance is zero

In [ ]:
def f(u):
    out = np.zeros(2)
    ### TODO: define f(z)
    raise NotImplementedError
    return np.array(out)
    
def dfdu(u):
    out = np.zeros((2,2))
    ### TODO: define f'(z)
    raise NotImplementedError
    return np.array(out)

## Attempt #1: Forward-Euler

Let's start simple - implement a single step of forward-Euler:

In [ ]:
def fe(ui, f, dfdu, dt):
    """
    Single step of forward-Euler.
    
    Args:
        ui (np.array): State at time-step i
        f (function): Fn f defining the ODE
        dfdu (function): Fn returning the derivative of f (not used in FE, just
                         for consistency with other methods)
        dt (float): Time-step
    Return:
        u (np.array): State at timestep i+1
    """
    ### TODO: implement forward-Euler
    raise NotImplementedError

Test your implementation with the following functions.  Fn `predict()` simulates the pendulum using a time-stepping function like `fe()`.  Fn `plot()` plots the time-history as computed by `predict()`.

In [ ]:
def predict(u_0, f, dfdu, T_final, N, step_fn):
    """
    Solve the ODE defined by f and dfdu in time, from initial condition `u_0`
    to final time `T_final` with `N` time-steps, using `step_fn()` to obtain
    $u_{i+1}$ from $u_i$ at each timestep.  Return array shaped (N+1, M)
    containing the solution u at each time, including t=0.
    """
    M = u_0.shape[0]
    dt = T_final/N
    u = np.zeros((N+1, M))
    u[0,:] = u_0
    for i in range(N):
        u[i+1,:] = step_fn(u[i,:], f, dfdu, dt)
    return u

def plot(u, T_final):
    """Plot the output of `predict()`."""
    N = u.shape[0]-1
    plt.plot(np.linspace(0,T_final,N+1), u[:,0]*180./np.pi, '-+')
    plt.xlabel('t')
    plt.ylabel('deg')

With an initial offset of $60^\circ$, choose a final time, and number of steps $N$ and simulate the pendulum with `predict()` and `plot()`:

In [ ]:
u_0 = np.array([60.*np.pi/180.,0])
T_final = 10.   # (s) Final time
N = 200       # (.) Number of time-steps
dt = T_final / N
# TODO: Run simulation with predict() and plot() results.
raise NotImplementedError

In order to check your solution, you should check if the result is physical - in partcular, if `alpha=0` (zero damping) energy is conserved, the pendulum amplitude should never change.  Is this the case in your simulation?  Check also for long times.  You should notice, that - even though your implementation *may* be correct - that the solution is always unphysical (growing in time) - though you might not notice if your `T_final` is not large enough!  To establish *why* - we need to analyse the stability of FE for this problem.

The function `plot_stability_region()` does just that, if you pass it a function as an argument that computes the amplification factor for your scheme.  E.g. for forward-Euler the function is: `def fe_stability(z): return 1+z`.  Test it.

In order to determine if forward-Euler is stable, in addition to the stability region, we need the eigenvalues of $\partial f/\partial u \cdot\Delta t$.  If these are all in the stable region of the scheme, the method is stable.  Complete `plot_eigvals()` to plot these values on top of the stability region.  Use `np.linalg.eigvals()` for each time-step of the output of `predict()`.  This will result in a chain of points, all of which should be in the stability region.

In [ ]:
def plot_stability_region(stabfn):
    """
    Given a stability fn for a particular scheme, plot the stability region. 
    Green is unstable, white is stable.
    
    Args:
      stabfn (function): Takes one complex argument, returns complex value. 
    """
    x = np.linspace(-4,4,100)
    X = np.meshgrid(x, x)
    z = X[0]+1j*X[1]
    Rlevel = np.abs(stabfn(z))
    plt.figure(figsize=(8,8))
    plt.contourf(x,x,Rlevel,[1,1000])
    plt.contour(x,x,Rlevel,[1,1000])
    plt.xlabel(r'Re'); plt.ylabel(r'Im')
    plt.plot([0,0],[-4,4], '-k')
    plt.plot([-4,4],[0,0], '-k')
    plt.axes().set_aspect('equal')
    
    
def plot_eigvals(u, dfdu, dt):
    """
    Plot the eigenvalues of $df/du \cdot \Delta t$, for all values of the solution u.
    
    Args:
      u (array (N, M)): Solution array (real-valued) with N timesteps, M values per step.
      dt (float): Timestep.
    Return:
      eigs (array (N, M)): Eigenvalue array (complex-valued) for each solution time.
    """
    N, M = u.shape
    eigs = np.zeros((N, M), dtype=np.complex)
    # TODO: Compute and plot eigenvalues
    raise NotImplementedError
    return eigs

Test your code by creating plots, and finally, by finding the largest amplificiation factor for `T_final = 10`, `N = 200`, and unchanged physcial conditions (i.e. with damping).  The maximum value should be about `1.0120701`.  As `N` gets larger this number should approach $1$ from above, but always be $>1$ (to machine precision!).

In [ ]:
def fe_stability(z): return 1+z

plot_stability_region(fe_stability)
eigs = plot_eigvals(u, dfdu, dt)

# Zoom at the origin as h -> 0
h = 4
plt.xlim(-h,h); plt.ylim(-h,h)

# TODO: Investigate the stability of FE for the pendulum

It looks like FE is never an option, at least for this equation.  So let's bring out the big guns!

## Attempt #2: Backward-Euler

You remember that backward-Euler is unconditionally stable - which, after the last section - seems like it should be a relief.  You decide to use Newton to solve the problem at each time-step - fortunately you still have the implementation of Newton's method from Assignment A, which (of course) you implemented in the most general multi-dimensional form (using `np.linalg.inv()` to invert the derivative matrix):

In [ ]:
def newton(f, dfdx, x_0, iter_max=10, min_error=1e-14):
    """
    Newton method for rootfinding.  It guarantees quadratic convergence 
    given f'(root) != 0 and abs(f'(xi)) < 1 over the domain explored.

    Args:
        f (function): function 
        dfdx (function): derivative of f
        x_0 (array M): starting guess
        iter_max (int): max number of iterations
        min_error (float): min allowed error

    Returns:
        x[-1] (float) = root of f
        x (np.array) = history of convergence
    """
    x = [x_0]
    for i in range(1,iter_max):
        xp1 = x[-1] - np.linalg.inv(dfdx(x[-1])).dot(f(x[-1]))
        x.append(xp1)
        if np.max(np.abs(f(x[-1]))) < min_error:
            break
    return x[-1], np.array(x)

Using the above, implement backward-Euler in the template below.  Bear in mind that at each time-step you are not looking for a root of $f$, but of a different function (say $\phi$) defined by the BE iteration.  Use the solution at the last time-step as the initial guess for Newton.  So we can analyse the stability, implement also `be_stability()` in analogy to `fe_stability()` above.

In [ ]:
def be(ui, f, dfdu, dt):
    """
    Single step of backward-Euler with Newton solution method.
    
    Args:
        ui (np.array): State at time-step i
        f (function): Fn f defining the ODE
        dfdu (function): Fn returning the derivative of f (not used in FE, just
                         for consistency with other methods)
        dt (float): Time-step
    Return:
        u (np.array): State at timestep i+1
    """
    # TODO: Implement backward-Euler using `newton()`
    raise NotImplementedError

def be_stability(z): 
    raise NotImplementedError

Use your code and `predict/plot()` to solve the ODE, and compare with forward-Euler.  With `alpha = 0` do you see physical behaviour from BE?  Do you see stable behaviour?  To verify your code, compare these results with the theory.  Also, your angle after 1 timestep with damping, and `dt = 0.1` should be about `55.97929 degrees`.

The code below also times the two methods - check also that this conforms to what you know about the methods.

In [ ]:
T_final = 10.     # (s) Final time
N = 200           # (.) Number of time-steps
dt = T_final / N
#alpha = 0.0

start = time()
u = predict(u_0, f, dfdu, T_final, N, fe)
print('FE CPU: %10.3g s' % (time()-start,))
plot(u, T_final)

start = time()
u = predict(u_0, f, dfdu, T_final, N, be)
print('BE CPU: %10.3g s' % (time()-start,))
plot(u, T_final)

Perform a stability analysis for BE as for FE before.  What is the largest amplification factor?

In [ ]:
plot_stability_region(be_stability)
eigs = plot_eigvals(u, dfdu, dt)

# Zoom at the origin as h -> 0
h = 4
plt.xlim(-h,h); plt.ylim(-h,h)

## Attempt #3: Runge-Kutta

Backward-Euler seems also disappointing - in other ways.  You decide to round out your toolbox with Runge-Kutta 4-stage (RK4), before moving on to the orbial mechanics.  Implement RK4, together with its corresponding `rk4_stability()` function.

In [ ]:
def rk4(u, f, dfdu, dt):
    """
    Runge-Kutta explicit 4-stage scheme - single step.
    
    Args:
        z (np.array): State at time-step i
        f (function): Fn f defining the ODE
        dt (float): Time-step
    Return:
        z (np.array): State at timestep i+1
    """
    # TODO: Implement RK4
    raise NotImplementedError    

def rk4_stability(z): 
    raise NotImplementedError

Again run the code to verify your implementation.  With the usual settings, and `dt = 0.1` after 1 step the solution should be `57.650555` degrees.

In [ ]:
T_final = 10.     # (s) Final time
N = 100           # (.) Number of time-steps
dt = T_final / N
u = predict(u_0, f, dfdu, T_final, N, rk4)
plot(u, T_final)

Finally the stability region should look like a blob with lobes, and the maximum amplification factor should be $<1$ but very close to $1$, for the above settings.

In [ ]:
plot_stability_region(rk4_stability)
eigs = plot_eigvals(u, dfdu, dt)
print np.max(np.abs(rk4_stability(eigs)))

# Zoom at the origin as h -> 0
h = 4
plt.xlim(-h,h); plt.ylim(-h,h)

## Finally: Orbital mechanics

You finally feel ready to tackle the original problem you were interested in.  Setting it up is a bit long-winded, but in summary:

Consider now $M$ point-masses mutually gravitating in 2d, with locations $(x_i,y_i) = \mathbf{x}_i \in \mathbb{R}^2$ and masses $m_i$.  Mass $j$ exerts a force due to gravity on mass $i$ of:
$$
F_{ij} = \frac{G m_i m_j}{r^2}\cdot \frac{\mathbf{x}_j-\mathbf{x}_i}{|\mathbf{x}_j-\mathbf{x}_i|}, \qquad r := |\mathbf{x}_j - \mathbf{x}_i|
$$
i.e. inverse-square law along the line connecting the masses.  Newton II applies:
$$
m_i \mathbf{x}_i'' = \sum_{j=0,j\neq i}^{M-1} F_{ij} =: F_i = (F_i^x, F_i^y)
$$
Again in canconical form $\mathbf{u}'=f(\mathbf{u})$, with
$$
\mathbf{u} = \begin{pmatrix} x_0,\dots x_{M-1}, y_0,\dots y_{M-1}, x'_0,\dots x'_{M-1}, y'_0,\dots y'_{M-1} \end{pmatrix}
$$
and
$$
f(\mathbf{u}) = \begin{pmatrix} x'_0,\dots x'_{M-1}, y'_0,\dots y'_{M-1},
F_0^x,\dots,F_{M-1}^x, F_0^y,\dots,F_{M-1}^y\end{pmatrix}.
$$

To model the Earth-moon system, we need the various constants:
$$
G = 6.67384\times 10^{-11}\,\mathrm{N}\cdot(\mathrm{m/kg})^2\\
$$
$$
m_\mathrm{earth} = 5.972\times 10^{24}\,\mathrm{kg},\quad m_\mathrm{moon} = 7.348 \times 10^{22}\,\mathrm{kg},\quad m_\mathrm{apollo}\simeq 30\times 10^3\,\mathrm{kg}.
$$
Moon's distance and velocity relative to Earth:
$$
x_\mathrm{moon}= 384400.0\times 10^3\,\mathrm{m},\quad y'_\mathrm{moon} = 1.023\times 10^3\,\mathrm{m/s}.
$$
To make the center-of-mass of the combined system fixed, choose a cancelling initial velocity for the Earth:
$$
\quad y'_\mathrm{earth} = -y'_\mathrm{moon} \frac{m_\mathrm{moon}}{m_\mathrm{earth}}
$$

This whole system is implemented in the following code for a general M-body problem:

In [ ]:
# TODO ... fill in your net_id
net_id = 'jordi'

n_body = 3
M_multibody = 4 * n_body
masses = [5.972e24,      # Mass of earth [kg]
          7.34767309e22, # Mass of moon [kg]
          70.0]          # ~ mass of you [kg]
G = 6.67384e-11     # Universal graviational constant

def f_multibody(u):
    x    = u[0:n_body]
    y    = u[n_body:2*n_body]
    xdot = u[2*n_body:3*n_body]
    ydot = u[3*n_body:]
    f = np.zeros((n_body,4))
    f[:,0] = xdot
    f[:,1] = ydot
    for i in range(n_body):
        for j in range(n_body):
            if i != j:
                r = np.sqrt( (x[j]-x[i])**2 + (y[j]-y[i])**2 )
                f[i,2] += G*masses[j]*(x[j]-x[i])/r**3
                f[i,3] += G*masses[j]*(y[j]-y[i])/r**3
    return f.T.flatten()

def dfdu_multibody(u):
    x    = u[0:n_body]
    y    = u[n_body:2*n_body]
    xdot = u[2*n_body:3*n_body]
    ydot = u[3*n_body:]
    jac = np.zeros((M_multibody,M_multibody))
    
    jac[0:2*n_body,2*n_body:] = np.eye(2 * n_body)
    for i in range(n_body):
        for j in range(n_body):
            if i != j:    
                r = np.sqrt( (x[j]-x[i])**2 + (y[j]-y[i])**2 )
                dFdxj = G*masses[j]*(np.eye(2)/r**3 - 
                          3./r**5*np.array([[(x[j]-x[i])**2,(x[j]-x[i])*(y[j]-y[i])],
                                            [(x[j]-x[i])*(y[j]-y[i]),(y[j]-y[i])**2]]))
                dFdxi = G*masses[j]*(-np.eye(2)/r**3 - 
                          3./r**5*np.array([[(x[j]-x[i])**2,(x[j]-x[i])*(y[j]-y[i])],
                                            [(x[j]-x[i])*(y[j]-y[i]),(y[j]-y[i])**2]]))                
                jac[2*n_body+i,j]        += dFdxj[0,0]
                jac[2*n_body+i,j]        += dFdxj[0,1]
                jac[2*n_body+i,n_body+j] += dFdxj[0,1]
                jac[3*n_body+i,j]        += dFdxj[1,0]
                jac[3*n_body+i,n_body+j] += dFdxj[1,1]
    return jac

x_moon = 384400.0e3   # [m]

# Calculate velocity of earth/moon resulting in a circular orbit
# about the Earth-moon CoM.
r_moon  = masses[0]*x_moon/(masses[0]+masses[1])
r_earth = masses[1]*x_moon/(masses[0]+masses[1])
T = np.sqrt(4.*np.pi**2*x_moon**3 / (G*(masses[0]+masses[1]))) # Orbital period
vel_moon  = 2.*np.pi*r_moon/T
vel_earth = -2.*np.pi*r_earth/T

from helpers import hash_string2int, rand
vel = 1850 + (2*rand(hash_string2int(net_id, 2**32), 20)[-1] - 1.)*200

def init_multibody():
    """Initial conditions for Earth-Moon system... and you!"""
    theta = 25.*np.pi/180
    #        Earth      Moon          You
    return [        0,   x_moon,     x_moon/4*np.cos(theta),    # x-position (Earth at origin)
                    0,        0,     x_moon/4*np.sin(theta),    # y-position
                    0,        0,     vel*-np.sin(theta),        # x-vel (center-of-mass stationary)
            vel_earth, vel_moon, vel*np.cos(theta)]             # y-velocity

u_0 = np.array(init_multibody())     # Initial condition

Apply your numerical methods to the Earth-moon system and do the usual sanity checks.  Verify that the period of the moon is about 28 days, and that the Earth-moon distance is approximately constant.

In [ ]:
# TODO: Compute your Earth-distance
T_final = ???
N = ???
u = predict(u_0, f_multibody, dfdu_multibody, T_final, N, ???)

plt.figure(figsize=(10,10))
plt.plot(u[:,0], u[:,0+n_body], '-or', label='Earth')
plt.plot(u[:,1], u[:,1+n_body], '-k', label='Moon')
plt.plot(u[:,2], u[:,2+n_body], '-g', label='You')
plt.xlim(-2.1*x_moon, 2.1*x_moon)
plt.ylim(-2.1*x_moon, 2.1*x_moon)
plt.legend()

To get the points for the assignment, replace 'jordi' with your netid.  Compute and report your distance from Earth (in meters) after 1 week ($24\times 7$ hours) as predicted by both RK4 and BE, both with a time-step of 5 minutes (don't worry if you fly off into the void).  For BE you should have a `iter_max=10`, and `min_error=1e-14` in Newton.

Jordi's distances with these settings are roughly:

 - BE:  $76091000$ m
 - RK4: $96149000$ m

Fill in your distances below:

In [ ]:
# TODO: Your distances
BE_dist = ???
RK4_dist = ???

## Special bonus point (lunar insertion)

+1 full (additional) grade bonus point will be awarded to the first student who emails me <r.p.dwight@tudelft.nl> with a convincing lunar insertion plan, consisting of the following:

- An initial stable approximately circular orbit around Earth at an "altitude" of 6556km (distance from the center of Earth).  Required is an analytic expression for the initial condition.
- The timing, power and duration of two burns: an initial burn to leave Earth orbit, and a final burn to enter lunar orbit.
- Each burn is to have a maximum acceleration of 0.1g and a maximum duration of 1.5 hours - at least one complete orbit of Earth should be performed before the first burn.
- Each burn can only be in the direction of travel, but may be positive or negative.
- The final lunar orbit must be approximately circular, stable, and close to the moon.  Specifically: for a period of 1 week after the final time the distance to the (center of the) moon should never be greater than $\frac{1}{20}x_{moon}$, and the orbital eccentricity $e < 0.1$.
- The insertion is achieved quickly (the final time may not be greater than 6 months).
- All this is true when assessed with RK4 with a 5 minute time-step, and a significantly smaller, stable time-step during the burns.

Code must be provided generating plots, which readily and clearly demonstrate that all above conditions are satisfied.

Good luck!

